In [1]:
# import dependencies
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [2]:
# read the PRS score features into DataFrame
df_prs = pd.read_csv('PRS.csv')
df_prs

,FID,PID,V1,V2,V3,V4,V5,V6,V7,V8,...,V1391,V1392,V1393,V1394,V1395,V1396,V1397,V1398,V1399,V1400
0,sample_1_0,sample_1_0,1.72650,1.72650,1.72650,1.72650,1.72650,1.72650,1.72650,1.72650,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,sample_1_1,sample_1_1,7.00240,7.00240,7.00240,7.00240,7.03260,7.03260,7.03260,7.03260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,sample_1_10,sample_1_10,6.02980,6.02980,6.02980,6.02980,6.06000,6.06000,6.06000,6.06000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,sample_1_100,sample_1_100,1.25400,1.25400,1.25400,1.25400,1.25400,1.25400,1.25400,1.25400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,sample_1_1000,sample_1_1000,-0.64570,-0.64570,-0.64570,-0.64570,-0.61550,-0.61550,-0.61550,-0.61550,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,sample_1_99995,sample_1_99995,4.79275,4.79275,4.79275,4.79275,4.82295,4.82295,4.82295,4.80600,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,sample_1_99996,sample_1_99996,2.26680,2.26680,2.26680,2.26680,2.29700,2.29700,2.29700,2.29700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,sample_1_99997,sample_1_99997,1.36000,1.36000,1.36000,1.36000,1.36000,1.36000,1.36000,1.36000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,sample_1_99998,sample_1_99998,3.44790,3.44790,3.44790,3.44790,3.44790,3.44790,3.44790,3.44790,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# split data into features X and targets y
n_subjects = len(df_prs.index)
y = np.random.randint(2, size=n_subjects) # randomly generate labels (0: control, 1:disease)
X = df_prs.iloc[:, 2:].to_numpy() # use PRS scores as features

# model training with a random forest classifier
clf = RandomForestClassifier()
clf.fit(X, y)

RandomForestClassifier()

In [4]:
# calculate the impurity-based importance of each feature/PRS
imp = clf.feature_importances_
df_imp = pd.DataFrame(columns=df_prs.columns[2:])
df_imp.loc[0] = imp
df_imp.to_csv('PRS_feature_importance.csv', index=False)

In [5]:
# save the model to disk/instance
pickle.dump(clf, open('model.sav', 'wb'))

In [7]:
# example usage of the model
load_clf = pickle.load(open('model.sav', 'rb'))
disease_label_idx = np.where(load_clf.classes_==1)[0][0] # find out which class is the disease class in the ouput class probabilities
pred_y_prob = load_clf.predict_proba(X) # n x 2 class probabilities for n input patients
df_prob = pd.DataFrame(pred_y_prob[:, disease_label_idx], columns=['disease_probability']) # the second column of output probability is the one for disease
df_prob.to_csv('disease_probability.csv')